# Nerf Studio on AWS Sagemaker

## Introduction
In this Jupyter notebook we are going to show a POC for a [NERF studio](https://docs.nerf.studio/en/latest/) built out entirely on AWS Sagemaker for end to end ML/OPS training and volumetric rendering. 

## Expected results 
At the end of this notebook, you will have generated a nerf of any video that captures a volumnetric space from at least 3 different perspectives, or depths of fields. Additionally you will also be able to use the same notebook to generate NERFs for any image of your chosing as well. 

## FAQs
- This Jupyter notbook is a mirror of [NERF Studio Project](https://colab.research.google.com/github/nerfstudio-project/nerfstudio/blob/main/colab/demo.ipynb#scrollTo=msVLprI4gRA4) that is available on Google Colab, but purpose built to work on AWS Sagemaker with a much larger capabilities and selection for underlying compute
- Please read the FAQs in the original Colab Notebook for more details about data processing and Training related concerns
- Please follow the instruction for each cell and carefully read them before running. You may have to restart the kernel a couple times.
- A lot of dependencies are purpose built for this Notebook and hence deviates to a good degree from the original Colab Notebook.

In [ ]:
# Make sure Cuda Toolkit is installed
! nvcc --version
# Make sure GPU is online
! nvidia-smi
# Make sure Cuda toolkit is in /usr/local/*
! whereis nvcc

In [2]:
#@markdown <h1>Install Nerfstudio and Dependencies (~8 min)</h1>
%mkdir -p /content
%cd /content/
%pip install --upgrade pip
%pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

# Installing TinyCuda
%cd /content/
!pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

# Installing COLMAP
%cd /content/
!apt-get install -y \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev
!apt-get install -y colmap

# Install nerfstudio
%cd /content/
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git

/content
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.
/content
  Cloning https://github.com/NVlabs/tiny-cuda-nn/ to /tmp/pip-req-build-5xpydjmo
  Running command git clone --filter=blob:none --quiet https://github.com/NVlabs/tiny-cuda-nn/ /tmp/pip-req-build-5xpydjmo
  Resolved https://github.com/NVlabs/tiny-cuda-nn/ to commit 28ca991f99b44d10387d73077c07ccfdd7f96275
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-filesystem-dev is already the newest version (1.71.0.0ubuntu2).
libboost-program-options-dev is already the newest version (1.71.0.0ubuntu2).
libboost-system-dev is already the newest version (1.71.0.0ubuntu2).
libmetis-dev is already the newes

In [ ]:
# Validate that COLMAP has installed successfully
! colmap -h

# Processing Data
## Upload your own images/video

In [15]:
# Only run this cell if you had to restart this notebook for some reason and do not need to re-process the data
import os
import glob
from IPython.display import display, HTML


In [5]:
import os
import glob
from IPython.display import display, HTML

scene = '\uD83C\uDFA5 upload your own video' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = ' '.join(scene.split(' ')[1:])

if scene in [ 'upload your own video']:
    display(HTML('<h3>Select your custom data</h3>'))
    display(HTML('<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>'))
    display(HTML('<p/>Note: This may take time, especially on higher resolution inputs, so we recommend to download dataset after creation.<p>'))
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    # Upload your video file to the appropriate directory in AWS SageMaker Studio
    # Replace the next line with the appropriate path of the uploaded file
    uploaded_files = ["/home/ec2-user/data/nerfstudio/custom_data/20230708_194635.mp4"]
    %cd /home/ec2-user/
    video_path = uploaded_files[0]
    !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/
    
    scene = "custom_data"

print("Data Processing Succeeded!")

/content/data/nerfstudio/custom_data
/home/ec2-user
Number of frames in video: 1952ages...
Number of frames to extract: 326
(     ●) Converting video to images...
[05:11:16] 🎉 Done converting video to images.                                                 ]8;id=544005;file:///usr/local/lib/python3.9/site-packages/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=601129;file:///usr/local/lib/python3.9/site-packages/nerfstudio/process_data/process_data_utils.py#182\182]8;;\
▄ Downscaling images...0m
[05:13:59] 🎉 Done downscaling images.                                                         ]8;id=95219;file:///usr/local/lib/python3.9/site-packages/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=132872;file:///usr/local/lib/python3.9/site-packages/nerfstudio/process_data/process_data_utils.py#380\380]8;;\
🌔  Running COLMAP feature extractor...0m
[05:14:15] 🎉 Done extracting COLMAP features.                      

In [6]:
# We also need to get the right version of nodejs and npm setup.. Without which a lot of the lower cells will fail. Please troubleshoot as needed based on the underlying compute. 
!curl -fsSL https://deb.nodesource.com/setup_19.x | bash - &&\
apt-get install -y nodejs


## Installing the NodeSource Node.js 19.x repo...


## Populating apt-get cache...

+ apt-get update
Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://deb.nodesource.com/node_19.x focal InRelease                     
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1072 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1371 kB]
Fetched 2779 kB in 1s (2462 kB/s)  
Reading package lists... Done

## Confirming "focal" is supported...

+ curl -sLf -o /dev/null 'https://deb.nodesource.com/node_19.x/dists/focal/Release'

## Adding the NodeSource signing key to your keyring...

+ 

In [7]:
!npm --version

9.8.0


In [14]:
#@markdown <h1>Set up and Start Viewer</h1>

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(5) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
print(lines)
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

/content
m#################⠂) ⠦ reify:yargs-parser: timing reifyNode:node_modules/localtK
changed 22 packages in 1s

3 packages are looking for funding
  run `npm fund` for details
['your url is: https://khaki-onions-arrive.loca.lt\n']
https://viewer.nerf.studio/?websocket_url=wss://khaki-onions-arrive.loca.lt
You may need to click Refresh Page after you start training!


In [16]:
#@markdown <h1>Start Training</h1>

%cd /content
if os.path.exists(f"data/nerfstudio/custom_data/transforms.json"):
    !ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data "data/nerfstudio/custom_data" --downscale-factor 4
else:
    from IPython.core.display import display, HTML
    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML('<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>'))

/content
2023-07-13 07:12:04.849711: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 07:12:04.991029: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2023-07-13 07:12:04.991125: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-07-13 07:12:04.991316: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [18]:
#@title # Render Video { vertical-output: true }
#@markdown <h3>Export the camera path from within the viewer, then run this cell.</h3>
#@markdown <h5>The rendered video should be at renders/output.mp4!</h5>


base_dir = "/content/outputs/unnamed/nerfacto/"
training_run_dir = base_dir + "2023-07-13_062816"

%cd $training_run_dir
!cp /root/data-manual-upload/camera_path.json .
!pwd
#uploaded = ["/root/data-manual-upload/camera_path.json"]
uploaded_camera_path_filename = "camera_path.json"

config_filename = training_run_dir + "/config.yml"
camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

%cd /content/
!ns-render camera-path --load-config $config_filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4

/content/outputs/unnamed/nerfacto/2023-07-13_062816
/content/outputs/unnamed/nerfacto/2023-07-13_062816
/content
2023-07-13 07:14:37.548254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 07:14:37.689286: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2023-07-13 07:14:37.689388: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-07-13 07:14:37.689578: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from 